import required packages

In [ ]:
import pathlib
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import xarray as xr

In [ ]:
config = {'inputDirectory':'', 
 'bands':[1,2,3],
 'sizeCutOut':20,
 'nEpochMax':2,
 'sizeStep':5}

In [ ]:
def create_input_list(inputDir):
    inputList = [t for t in pathlib.Path(inputDir).glob('*.tif')]
    return inputList

In [ ]:
def read_config(config):
    if pathlib.Path(config).is_file():
        raise NotImplementedError()
        
    else :
        inputDir = config['inputDirectory']
        sizeCutOut = config['sizeCutOut']
        nEpochMax = config['nEpochMax']
        sizeStep = config['sizeStep']
        
    return inputDir, sizeCutOut, nEpochmax, sizeStep

define Dataset class and methods

In [ ]:
class Dataset:
    
    def __init__(self, tile_list, cutout_size, bands, offset=0, stride=None, num_tiles=None, shuffle_tiles=False):
        self.cutout_size = cutout_size
        self.bands = bands
        self.stride = stride if stride is not None else self.cutout_size
        _num_tiles = num_tiles if num_tiles is not None else len(tile_list)
        self.tiles = random.sample(tile_list, _num_tiles) if shuffle_tiles else tile_list[:_num_tiles]
        if offset >= cutout_size:
            raise ValueError(
                "offset larger than window size - set "
                "offset to {}".format(sizeCutOut%offset)
            )
        self.offset = offset
    
    def set_mask(self, shapefile, invert=False):
        """ Mask a selection of the pixels using a shapefile. """
        raise NotImplementedError()
    
    def to_tf(self):
        """ Obtain dataset as a tensorflow `Dataset` object. """
        ds = tf.data.Dataset.from_generator(
            self._generate_cutouts,  
            output_types=(tf.float64, tf.float64, tf.float32), 
            output_shapes=(
                None,  # x
                None,  # y
                (None, None, self.cutout_size, self.cutout_size)  # samples, bands, x_win, y_win
            )
        )
        return ds.flat_map(lambda x,y,z: tf.data.Dataset.from_tensor_slices((x,y,z)))

    def _generate_cutouts(self):
        """ 
        Iterate over (a selection of) the tiles yielding all 
        cutouts for each of them.
        """
        for tile in self.tiles:
            
            print(f"Reading tile {tile}!")
            
            # read tile
            da = xr.open_rasterio(tile)
            
            # TODO: apply mask here?
            
            # apply offset
            da = da.shift(x=self.offset, y=self.offset)  # only shift data, not coords
            da['x'] = da.x.shift(x=self.offset)
            da['y'] = da.y.shift(y=self.offset)

            # generate windows
            da = da.rolling(x=self.cutout_size, y=self.cutout_size)
            da = da.construct({'x': 'x_win', 'y': 'y_win'}, stride=self.stride)

            # drop NaN-containing windows
            da = da.stack(sample=('x', 'y'))
            da = da.dropna(dim='sample', how='any')
            yield (da.sample.coords['x'], 
                   da.sample.coords['y'], 
                   da.data.transpose(3, 0, 1, 2))  # samples, bands, x_win, y_win

### Begin feedloop

In [ ]:
inputDir, sizeCutOut, nEpochmax, sizeStep = read_config(config)

In [ ]:
inputList = create_input_list(inputDir)

In [ ]:
epochCounter = 0

In [ ]:
while epochcounter < nEpochsMax:
    offset = epochcounter*sizeStep
    
    inData = Dataset(inputList,sizeCutOut,offset=offset)
    dataSet = inData.to_tf()
    dataSet.shuffle()